In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from itertools import product
import numpy as np
import re
import json
import io
import string

In [2]:
# import charity commission json file as pandas df

with io.open(r"C:\Users\Elisha.Zissman\OneDrive\Data Projects\EPC\combining\publicextract.charity.json", 'r', encoding='utf-8-sig') as json_file:  
    data = json.load(json_file)

cc = pd.DataFrame(data)

In [3]:
# import epc file as pandas df

epc = pd.read_excel(r"C:\Users\Elisha.Zissman\OneDrive\Data Projects\EPC\combining\epc_community.xlsx")

In [4]:
# show epc columns

list(epc)

['Unnamed: 0.1',
 'Unnamed: 0',
 'LMK_KEY',
 'ADDRESS1',
 'ADDRESS2',
 'ADDRESS3',
 'Postcode',
 'BUILDING_REFERENCE_NUMBER',
 'ASSET_RATING',
 'ASSET_RATING_BAND',
 'PROPERTY_TYPE',
 'INSPECTION_DATE',
 'LOCAL_AUTHORITY',
 'CONSTITUENCY',
 'COUNTY',
 'LODGEMENT_DATE',
 'TRANSACTION_TYPE',
 'NEW_BUILD_BENCHMARK',
 'EXISTING_STOCK_BENCHMARK',
 'BUILDING_LEVEL',
 'MAIN_HEATING_FUEL',
 'OTHER_FUEL_DESC',
 'SPECIAL_ENERGY_USES',
 'RENEWABLE_SOURCES',
 'FLOOR_AREA',
 'STANDARD_EMISSIONS',
 'TARGET_EMISSIONS',
 'TYPICAL_EMISSIONS',
 'BUILDING_EMISSIONS',
 'AIRCON_PRESENT',
 'AIRCON_KW_RATING',
 'ESTIMATED_AIRCON_KW_RATING',
 'AC_INSPECTION_COMMISSIONED',
 'BUILDING_ENVIRONMENT',
 'ADDRESS',
 'LOCAL_AUTHORITY_LABEL',
 'CONSTITUENCY_LABEL',
 'POSTTOWN',
 'LODGEMENT_DATETIME',
 'PRIMARY_ENERGY_VALUE',
 'UPRN',
 'UPRN_SOURCE',
 'community building?']

In [6]:
# show charity commission columns

list(cc)

['date_of_extract',
 'organisation_number',
 'registered_charity_number',
 'linked_charity_number',
 'charity_name',
 'charity_type',
 'charity_registration_status',
 'date_of_registration',
 'date_of_removal',
 'charity_reporting_status',
 'latest_acc_fin_period_start_date',
 'latest_acc_fin_period_end_date',
 'latest_income',
 'latest_expenditure',
 'charity_contact_address1',
 'charity_contact_address2',
 'charity_contact_address3',
 'charity_contact_address4',
 'charity_contact_address5',
 'charity_contact_postcode',
 'charity_contact_phone',
 'charity_contact_email',
 'charity_contact_web',
 'charity_company_registration_number',
 'charity_insolvent',
 'charity_in_administration',
 'charity_previously_excepted',
 'charity_is_cdf_or_cif',
 'charity_is_cio',
 'cio_is_dissolved',
 'date_cio_dissolution_notice',
 'charity_activities',
 'charity_gift_aid',
 'charity_has_land']

In [7]:
# concatenate charity address into one column

# Assuming cc is a pandas DataFrame
cc['ADDRESS'] = cc[['charity_contact_address1', 'charity_contact_address2', 'charity_contact_address3', 'charity_contact_address4', 'charity_contact_address5']].fillna('').apply("-".join, axis=1)


In [8]:
# rename postcode to match epc 

cc = cc.rename(columns={'charity_contact_postcode':'Postcode'})

In [10]:
# exclude charities that are removed

cc = cc[cc['charity_registration_status'] != 'Removed']

In [15]:
# all epc certs

epc_all = pd.read_csv(r"C:\Users\Elisha.Zissman\OneDrive\Data Projects\EPC\combining\epc_data.csv")

In [17]:
# rename postcode

epc_all = epc_all.rename(columns={'POSTCODE':'Postcode'})

In [19]:
# merge and score the matches

merged_df2 = pd.merge(epc_all, cc, on='Postcode', how='inner')  # Change 'inner' to 'left' or 'right' if needed

# Define a function to calculate the fuzzywuzzy token ratio score
def calculate_fuzzy_ratio2(row):
    return fuzz.token_set_ratio(row['ADDRESS_x'], row['ADDRESS_y'])

# Calculate the fuzzywuzzy token ratio score for each row in the merged dataframe
merged_df2['Score'] = merged_df2.apply(calculate_fuzzy_ratio2, axis=1)

merged_df2['Score'] = pd.to_numeric(merged_df2['Score'], errors='coerce')

# Filter the merged dataframe to include only rows where the score is greater than 70
result_df = merged_df2[merged_df2['Score'] > 70]


In [25]:
# only results with 100% match

result_100 = result_df[result_df['Score'] == 100]

In [29]:
MILES_FOR_MEN = result_100[result_100['charity_name'] == 'MILES FOR MEN']

In [27]:
list(result_100)

['Unnamed: 0',
 'LMK_KEY',
 'ADDRESS1',
 'ADDRESS2',
 'ADDRESS3',
 'Postcode',
 'BUILDING_REFERENCE_NUMBER',
 'ASSET_RATING',
 'ASSET_RATING_BAND',
 'PROPERTY_TYPE',
 'INSPECTION_DATE',
 'LOCAL_AUTHORITY',
 'CONSTITUENCY',
 'COUNTY',
 'LODGEMENT_DATE',
 'TRANSACTION_TYPE',
 'NEW_BUILD_BENCHMARK',
 'EXISTING_STOCK_BENCHMARK',
 'BUILDING_LEVEL',
 'MAIN_HEATING_FUEL',
 'OTHER_FUEL_DESC',
 'SPECIAL_ENERGY_USES',
 'RENEWABLE_SOURCES',
 'FLOOR_AREA',
 'STANDARD_EMISSIONS',
 'TARGET_EMISSIONS',
 'TYPICAL_EMISSIONS',
 'BUILDING_EMISSIONS',
 'AIRCON_PRESENT',
 'AIRCON_KW_RATING',
 'ESTIMATED_AIRCON_KW_RATING',
 'AC_INSPECTION_COMMISSIONED',
 'BUILDING_ENVIRONMENT',
 'ADDRESS_x',
 'LOCAL_AUTHORITY_LABEL',
 'CONSTITUENCY_LABEL',
 'POSTTOWN',
 'LODGEMENT_DATETIME',
 'PRIMARY_ENERGY_VALUE',
 'UPRN',
 'UPRN_SOURCE',
 'date_of_extract',
 'organisation_number',
 'registered_charity_number',
 'linked_charity_number',
 'charity_name',
 'charity_type',
 'charity_registration_status',
 'date_of_registrati

In [28]:
result_100['charity_name'].unique()

array(['MILES FOR MEN', 'FC HARTLEPOOL',
       'P.R.O.P. - PEOPLES RELIEF OF PRESSURE', ...,
       'CWM TAF MORGANNWG MIND', '3GS DEVELOPMENT TRUST',
       'FOOTSTEPS TO FAMILY CONNECTIONS'], dtype=object)

In [30]:
MILES_FOR_MEN

,Unnamed: 0,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,Postcode,BUILDING_REFERENCE_NUMBER,ASSET_RATING,ASSET_RATING_BAND,PROPERTY_TYPE,...,charity_previously_excepted,charity_is_cdf_or_cif,charity_is_cio,cio_is_dissolved,date_cio_dissolution_notice,charity_activities,charity_gift_aid,charity_has_land,ADDRESS_y,Score
48,1342,ae3e91c8672f18118c5b8209ad37bb15763c66adb87a4e...,MIDDLETON GRANGE SHOPPING CENTRE,MIDDLETON GRANGE SHOPPING CENTRE,HARTLEPOOL,TS24 7RZ,10000859357,100,D,A1/A2 Retail and Financial/Professional services,...,False,None,False,False,None,For the public benefit provide facilities and ...,False,True,UNIT 135-Middleton Grange Shopping centre-HART...,100


In [31]:
FC_HARTLEPOOL = result_100[result_100['charity_name'] == 'FC HARTLEPOOL']

In [32]:
FC_HARTLEPOOL

,Unnamed: 0,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,Postcode,BUILDING_REFERENCE_NUMBER,ASSET_RATING,ASSET_RATING_BAND,PROPERTY_TYPE,...,charity_previously_excepted,charity_is_cdf_or_cif,charity_is_cio,cio_is_dissolved,date_cio_dissolution_notice,charity_activities,charity_gift_aid,charity_has_land,ADDRESS_y,Score
133,222,139050080962018102914230012200190,NaN,Censis,66 Church Street,TS24 7DN,918488240000,82,D,A1/A2 Retail and Financial/Professional services,...,False,None,True,False,None,The object of the CIO is the promotion of comm...,True,True,Salaam Shaheen C/O CENSIS-66 CHURCH STREET-HAR...,100


In [33]:
A_3GS_DEVELOPMENT_TRUST = result_100[result_100['charity_name'] == '3GS DEVELOPMENT TRUST']

In [34]:
A_3GS_DEVELOPMENT_TRUST

,Unnamed: 0,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,Postcode,BUILDING_REFERENCE_NUMBER,ASSET_RATING,ASSET_RATING_BAND,PROPERTY_TYPE,...,charity_previously_excepted,charity_is_cdf_or_cif,charity_is_cio,cio_is_dissolved,date_cio_dissolution_notice,charity_activities,charity_gift_aid,charity_has_land,ADDRESS_y,Score
384048,1274491,156345270022020072410142807230210,NaN,15 Chestnut Way,NaN,CF47 9SB,100479740000,67,C,B1 Offices and Workshop businesses,...,False,None,False,False,None,We are the main delivery organisation for the ...,False,True,3 G's Development Trust-15 Chestnut Way-MERTHY...,100


In [35]:
result_100.to_excel('newres.xlsx')